In [51]:
'''
    this code performs anomalies detection
'''

'\n    this code performs anomalies detection\n'

In [52]:
import pycaret
import pandas as pd
import os
import  matplotlib.pyplot as plt
import numpy as np
from pycaret.anomaly import *
from collections import defaultdict
from sklearn.model_selection import train_test_split


In [53]:
'''
    reads a file from which None rows are removed
'''

filePath=".//data//"
fileName="data_sample.csv"
modelPath=".//model//"
resultPath=".//results//"

'''
    If below variable is one then it trains a model from scratch
    otherwise it just do prediction from already trained model
'''
trainModels=1


In [54]:
    def readData():
        df1=pd.read_csv(filePath+fileName)
        print("\n\t shape:",df1.shape)

        return df1

In [55]:
df1=readData()
df1.head()



	 shape: (106749, 3)


,timeUTC,wind_speed,wind_direction
0,2014-12-19 14:40:00,4.94,138.0
1,2014-12-19 14:50:00,4.78,139.2
2,2014-12-19 15:00:00,4.88,140.6
3,2014-12-19 15:10:00,4.81,139.4
4,2014-12-19 15:20:00,4.76,137.3


In [56]:
dAnomoly=defaultdict(int)
train, test = train_test_split(df1, test_size=0.05, shuffle=False)
print("\n\t 1.train data:",len(train))
print("\n\t test data:",len(test))
#iforest=createModel()  

train = setup(train, normalize=True, ignore_features=['timeUTC'], session_id=124)  #
print("\n\t 2.train data:",len(train))

,Description,Value
0,session_id,124
1,Original Data,"(101411, 3)"
2,Missing Values,True
3,Numeric Features,2
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(101411, 2)"
9,CPU Jobs,-1



	 2.train data: 36


In [57]:
def count(temp,dAnomoly):
    exceptionCount=0
    #dAnomoly=defaultdict(int)
    for indx, row in temp.iterrows():
        
        try:
            if row["Anomaly"]==1:
                #print("\n\t row :",row )
                key=str([row["timeUTC"],row["wind_speed"],row["wind_direction"]])
                #row = tuple(row["timeUTC"],row["wind_speed"],row["wind_direction"])
                #print("\n\t key:",key)
                dAnomoly[key] += 1
        except Exception as e:
            exceptionCount+=1
            #print("\n\t exception is:",e)
            
    print("\n\t exceptionCount:",exceptionCount)
    
    #print("\n\t number of rows:",indx)
    return dAnomoly

#dAnomolyTrain=defaultdict(int)
#dAnomolyTest=defaultdict(int)

print("\n\t train data:",len(train))
print("\n\t test data:",len(test))


	 train data: 36

	 test data: 5338


In [58]:
'''
    train part isolation forest
'''
trainModels=1
if trainModels==1:
    print("\n\t 1.training model from scratch!!!")
    iforest = create_model('iforest')
    save_model(iforest,modelPath+'iforest')

elif os.path.isfile(modelPath+'iforest.pkl'):
    '''
        loads model if already present
    '''
    
    print("\n\t 1.using pretrained model training model")
    saved_iforest = load_model(modelPath+'iforest')

else:
    print("\n\t model absent please set trainModels=1 to retrain")

'''
    prediction part
'''

if trainModels==1:
    
    '''
        prediction on train data
    '''
    outlier_results_iforest=assign_model(iforest)
    outlier_results_iforest.to_csv(resultPath+"outlier_results_iforest_train.csv",index=False)

    '''
        prediction on test data
    '''
    unseen_predictions = predict_model(iforest, data=test)
    #unseen_predictions.head()
    #input("check")
    unseen_predictions.insert(0,"timeUTC",test["timeUTC"],True)
    unseen_predictions.to_csv(resultPath+"outlier_results_iforest_test_predictions.csv",index=False)
else:
    
    outlier_results_iforest = assign_model(saved_iforest)#predict_model(saved_iforest, data=test)
    outlier_results_iforest.to_csv(resultPath+"outlier_results_iforest_train.csv",index=False)
    
    
    '''
        prediction on test data
    '''
    unseen_predictions = predict_model(saved_iforest, data=test)
    unseen_predictions.insert(0,"timeUTC",test["timeUTC"],True)
    #unseen_predictions.head()
    unseen_predictions.to_csv(resultPath+"outlier_results_iforest_test_predictions.csv",index=False)
    
print("\n\t train prediction size:",outlier_results_iforest.shape)
print("\n\t test prediction size:",unseen_predictions.shape)
    
    

Transformation Pipeline and Model Succesfully Saved

	 train prediction size: (101411, 5)

	 test prediction size: (5338, 5)


In [59]:
dAnomolyTrain=defaultdict(int)


dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
len(dAnomolyTrain.keys())



	 exceptionCount: 0


5071

In [60]:
#len(unseen_predictions)

dAnomolyTest=defaultdict(int)
dAnomolyTest=count(unseen_predictions,dAnomolyTest)

print("\n\t dict length:",len(dAnomolyTest.keys()))


	 exceptionCount: 0

	 dict length: 590


In [61]:

def modeltrain(classifierName,trainModels):
    '''
        train part isolation forest
    '''
    #trainModels=1
    if trainModels==1:
        print("\n\t training model from scratch!!!:",classifierName)
        iforest = create_model(classifierName)
        save_model(iforest,modelPath+classifierName)

    elif os.path.isfile(modelPath+classifierName+'.pkl'):
        '''
            loads model if already present
        '''

        print("\n\t 1.using pretrained model training model")
        saved_iforest = load_model(modelPath+classifierName)

    else:
        print("\n\t model absent please set trainModels=1 to retrain")

    '''
        prediction part
    '''

    if trainModels==1:

        '''
            prediction on train data
        '''
        outlier_results_iforest=assign_model(iforest)
        outlier_results_iforest.to_csv(resultPath+"outlier_results_"+classifierName+"_train.csv",index=False)

        '''
            prediction on test data
        '''
        unseen_predictions = predict_model(iforest, data=test)
        #unseen_predictions.head()
        #input("check")
        unseen_predictions.insert(0,"timeUTC",test["timeUTC"],True)
        unseen_predictions.to_csv(resultPath+"outlier_results_"+classifierName+"_test_predictions.csv",index=False)
    else:

        outlier_results_iforest = assign_model(saved_iforest)#predict_model(saved_iforest, data=test)
        outlier_results_iforest.to_csv(resultPath+"outlier_results_"+classifierName+"_train.csv",index=False)


        '''
            prediction on test data
        '''
        unseen_predictions = predict_model(saved_iforest, data=test)
        unseen_predictions.insert(0,"timeUTC",test["timeUTC"],True)
        #unseen_predictions.head()
        unseen_predictions.to_csv(resultPath+"outlier_results_"+classifierName+"_test_predictions.csv",index=False)
    
    print("\n\t train prediction size:",outlier_results_iforest.shape)
    print("\n\t test prediction size:",unseen_predictions.shape)
    
    return outlier_results_iforest,unseen_predictions
    

In [62]:
trainModels=0

dAnomolyTrain=defaultdict(int)
dAnomolyTest=defaultdict(int)

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('iforest',trainModels)    
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('knn',trainModels)    
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('cluster',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('abod',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('histogram',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('lof',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('pca',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))
#######################################

#######################################
outlier_results_iforest,unseen_predictions=modeltrain('svm',trainModels)
dAnomolyTrain=count(outlier_results_iforest,dAnomolyTrain)
print("\n\t 1.dict length:",len(dAnomolyTrain.keys()))
dAnomolyTest=count(unseen_predictions,dAnomolyTest)
print("\n\t 2.dict length:",len(dAnomolyTest.keys()))


dictFrame = pd.DataFrame.from_dict(dAnomolyTrain.items())
#dictFrame.to_csv(resultPath+"dAnomolyTrain.csv")

dictFrame = pd.DataFrame.from_dict(dAnomolyTest.items())
#dictFrame.to_csv(resultPath+"dAnomolyTest.csv")




	 1.using pretrained model training model
Transformation Pipeline and Model Successfully Loaded

	 train prediction size: (101411, 5)

	 test prediction size: (5338, 5)

	 exceptionCount: 0

	 1.dict length: 5071

	 exceptionCount: 0

	 2.dict length: 590

	 1.using pretrained model training model
Transformation Pipeline and Model Successfully Loaded

	 train prediction size: (101411, 5)

	 test prediction size: (5338, 5)

	 exceptionCount: 0

	 1.dict length: 7514

	 exceptionCount: 0

	 2.dict length: 780

	 1.using pretrained model training model
Transformation Pipeline and Model Successfully Loaded

	 train prediction size: (101411, 5)

	 test prediction size: (5338, 5)

	 exceptionCount: 0

	 1.dict length: 9321

	 exceptionCount: 0

	 2.dict length: 909

	 1.using pretrained model training model
Transformation Pipeline and Model Successfully Loaded

	 train prediction size: (101411, 5)

	 test prediction size: (5338, 5)

	 exceptionCount: 0

	 1.dict length: 9321

	 exceptionCou

In [69]:
'''
    here I check how many time particular sample is counted by outlier classifie
    New column named count is added to store that count
'''

#outlier_results_iforest,unseen_predictions

def outlierVoting(df,d):
    
    df["vote"]=0
    
    for indx,row in df.iterrows(): 
        
        #print("\n\t row:",row)
        key=str([row["timeUTC"],row["wind_speed"],row["wind_direction"]])
        
        if key in d.keys():
            df.loc[indx,"vote"]=d[key]
        else:
            df.loc[indx,"vote"]=0
        
#         if indx>100:
#             break
    
    return df
    
'''
    training data
'''
df=outlierVoting(outlier_results_iforest,dAnomolyTrain)

df.to_csv(resultPath+"outlierVoting.csv",index=False)

'''
    test data
'''
#outlierVoting(unseen_predictions,dAnomolyTest)


'\n    test data\n'